The economic model uses the latest econonic data from Belgium to estimate the impact of the pandemic.

It is to be run in parallel to the SEIRS Network model and could easily be merged into it. 

In [ ]:
import numpy as np
import economic_model

In [ ]:
EcoMod = economic_model.EconomicModel
EcoMod.LoadInputs(EcoMod)

The network of individuals can be assigned to an occupation and a state.
The occupation will be any of the following, based on demographic data and 2018 employment census:
* [-2] Student (0-20 yo)
* [-1] Retired (70+ yo)
* [0] Non-working (active population that do not work or seek employment)
* [1 - 37] Working or work seeker in one of 37 economical sectors

The state of non-working population can be:
* [-1] Confined
* [0] Unconfined
And the state of working population can be:
* [1] Working from home
* [2] Mix of working from home and at work (office/factory/etc.)
* [3] Working at work (office/factory/etc.)
* [4] Temporarily unemployed
* [5] Absent

The function AssignOccupation returns a Status Matrix that assigns both an occupation and a state of work randomly, according to current data.

In [ ]:
PopulationMatrix = EcoMod.AssignOccupation(EcoMod,N = 2000, ConfinementPolicy = True)
print(PopulationMatrix[1:10,:])

Occupation information about the population can be extracted with the function ComputeOccupation. The number of confined, unconfined, working and unemployed individuals are returned. 

In [ ]:
print(EcoMod.ComputeOccupation(EcoMod,PopulationMatrix))

In order to assess different scenarios, it is possible to alter restrictions on selected sectors of the population with the function ChangeConfinementPolicy. 
A call to ChangeConfinementPolicy allows to set non-working individuals (students, retired, non-workers) to Confined or Unconfined status or to set a specific economy sector to a new confinement policy, between [0 - 1]. A policy of 0 maximizes the work from home and unemployment while a policy of 1 sends everyone to work (office/factory/etc.). Any number in between offers a linear tradeof.

For example, lifting the confinement on children and students can be done as follow:

In [ ]:
PopulationMatrix = EcoMod.ChangeConfinementPolicy(EcoMod,PopulationMatrix,-2, 1)
print(EcoMod.ComputeOccupation(EcoMod,PopulationMatrix))

Then opening back all retails (economic sector number 19):

In [ ]:
PopulationMatrix = EcoMod.ChangeConfinementPolicy(EcoMod,PopulationMatrix,19, 1)
print(EcoMod.ComputeOccupation(EcoMod,PopulationMatrix))

State of development of the Economic Model:

The model is still very basic, with only one objective function (ComputeOccupation). However, new objective functions taking into account added value of work can already be written as the information is readily available in the LoadInputs function.

Input-Output tables are available in the DATA folder, but not loaded in the model. 

No modification to the SEIRS model has been made to couple it with the economic model. However, as of today (April 27th), the link can be made by adding two attributes to each individual in the SEIRS: Occupation and Status. 
